In [1]:
import tensorflow as tf
import model_gen
import os
import configurations
from preprocessing import Label
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
import math
import matplotlib.pyplot as plt
import utility
import cv2

In [2]:
gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
config = configurations.Configuration()
label = Label()
util = utility.Util()

In [7]:
column_names = []
for i in range(self.config.KEYPOINT_COUNT + 1):
    column_names.append("x_{0}".format(i + 1))
    column_names.append("y_{0}".format(i + 1))

None


In [4]:
df = util.get_dataframe()

In [5]:
df.head()

,file_names,0,1,2,3,4,5,6,7,8,...,24,25,26,27,28,29,30,31,32,33
0,000001163.jpg,102.0,177.0,104.0,155.0,106.0,121.0,114.0,123.0,117.0,...,101.0,77.0,118.0,78.0,117.0,90.0,120.0,104.0,116.0,113.0
1,000003072.jpg,129.0,167.0,128.0,137.0,125.0,110.0,135.0,110.0,134.0,...,121.0,68.0,137.0,67.0,141.0,87.0,149.0,96.0,132.0,104.0
2,000004812.jpg,164.0,206.0,162.0,163.0,162.0,109.0,159.0,110.0,160.0,...,190.0,138.0,184.0,130.0,176.0,161.0,170.0,191.0,168.0,150.0
3,000013469.jpg,0.0,0.0,0.0,0.0,42.0,156.0,24.0,157.0,0.0,...,55.0,67.0,17.0,62.0,24.0,112.0,37.0,99.0,46.0,111.0
4,000015774.jpg,106.0,85.0,106.0,123.0,111.0,136.0,120.0,137.0,121.0,...,102.0,114.0,135.0,114.0,145.0,135.0,149.0,159.0,112.0,115.0


In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=df,
    directory=os.path.join(config.IMAGES_BASE_DIR, "train"),
    x_col="file_names",
    y_col=util.get_column_names(),
    target_size=config.INPUT_DIMS[:2],
    class_mode="multi_output",
    batch_size=config.BATCH_SIZE
)

In [ ]:
STEPS_PER_EPOCH = config.LEN_TRAIN_IMGS / config.BATCH_SIZE

In [ ]:
STEPS_PER_EPOCH = math.floor(STEPS_PER_EPOCH) + 1

In [ ]:
model_gen = model_gen.Model()

In [ ]:
model = model_gen.create_model_with_vgg16()

In [ ]:
model.summary()

In [ ]:
model_gen.compile(model)

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=STEPS_PER_EPOCH,
    epochs=config.EPOCHS
)

In [ ]:
model.save("model_v2")

In [ ]:
history.history.keys()

In [ ]:
loss = history.history["loss"]
mae = history.history["mean_absolute_error"]
mse = history.history["mean_squared_error"]
epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, 'bo', label='Loss')
plt.plot(epochs, mae, 'b', label='Mean Absolute Error')
plt.plot(epochs, mse, 'r', label='Mean Squared Error')
plt.title('History')
plt.legend()
plt.show()


In [ ]:
img_ids = os.listdir(os.path.join(config.IMAGES_BASE_DIR, "test"))[:config.BATCH_SIZE]

In [ ]:
imgs = []
for id in img_ids:
    imgs.append(util.imread_from_id(id))

In [ ]:
resized = []
for img in imgs:
    resized.append(cv2.resize(img, config.INPUT_DIMS[:2]))

In [ ]:
resized_tensor = np.array(resized)

In [ ]:
y_pred = model.predict(resized_tensor)

In [ ]:
y_pred[0] / 3

In [ ]:
plt.imshow(util.draw_keypoints(imgs[0], y_pred[0] / 3, 5))

In [ ]:
plt.imshow(util.to_rgb(resized[8]))